In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/lepidoptera"

early_regex = r"^.*-(early)$"
unidentified_regex = r"^.*-(spp|genera|genera-spp)$"
early_or_unidentified_regex = r"^.*-(early|spp|genera|genera-spp)$"

# Create datasets

In [4]:
if os.path.exists(f"{dataset_dir}/data"):
    shutil.rmtree(f"{dataset_dir}/data")
os.makedirs(f"{dataset_dir}/data")
for src_dir in [f"{dataset_dir}/../moth/data", f"{dataset_dir}/../butterfly/data"]:
    for class_dir in os.listdir(src_dir):
        shutil.copytree(f"{src_dir}/{class_dir}", f"{dataset_dir}/data/{class_dir}")

In [5]:
if os.path.exists(f"{dataset_dir}/val"):
    shutil.rmtree(f"{dataset_dir}/val")
os.makedirs(f"{dataset_dir}/val")
for src_dir in [f"{dataset_dir}/../moth/val", f"{dataset_dir}/../butterfly/val"]:
    for class_dir in os.listdir(src_dir):
        shutil.copytree(f"{src_dir}/{class_dir}", f"{dataset_dir}/val/{class_dir}")

# Count

In [2]:
classes = { class_dir: len([ img for img in os.listdir(f"{dataset_dir}/data/{class_dir}") ]) for class_dir in os.listdir(f"{dataset_dir}/data") }
early_classes = { class_name: count for class_name, count in classes.items() if re.match(early_regex, class_name) }
unidentified_classes = { class_name: count for class_name, count in classes.items() if re.match(unidentified_regex, class_name) }
print(f"Total Class count : {len(classes):6} ( Unidentified: {len(unidentified_classes):6} / Early-stage: {len(early_classes):6} / Identified-adult: {len(classes) - len(unidentified_classes) - len(early_classes):6} )")
print(f"Total  Data count : {sum(classes.values()):6} ( Unidentified: {sum(unidentified_classes.values()):6} / Early-stage: {sum(early_classes.values()):6} / Identified-adult: {sum(classes.values()) - sum(unidentified_classes.values()) - sum(early_classes.values()):6} )")

Total Class count :   5550 ( Unidentified:    446 / Early-stage:   1027 / Identified-adult:   4077 )
Total  Data count : 324034 ( Unidentified:  12202 / Early-stage:  25105 / Identified-adult: 286727 )


In [3]:
img2_class = []
img5_class = []
for class_dir in os.listdir(f"{dataset_dir}/data"):
    if not re.match(early_or_unidentified_regex, class_dir):
        img_cnt = sum([1 for file in os.listdir(f"{dataset_dir}/data/{class_dir}")])
        img2_class += [class_dir] if img_cnt <= 2 else []
        img5_class += [class_dir] if img_cnt <= 5 else []
print(f"{len(img2_class):6} classes with <=2 images")
print(f"{len(img5_class):6} classes with <=5 images")

   195 classes with <=2 images
   526 classes with <=5 images


In [4]:
generas = set()
for class_name in classes:
    generas.add(class_name.split('-')[0])
print(f"Genera count: {len(generas)}")

Genera count: 1866


# Train

### Model A (resnet-152)

In [16]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet152", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.ta.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.1822 Acc: 0.5864  | Val Loss: 1.4265 Acc: 0.6265  | Elapsed time: 1:21:41.822510
Epoch    2 /    5  | Train Loss: 0.8145 Acc: 0.8010  | Val Loss: 1.2405 Acc: 0.7531  | Elapsed time: 2:40:29.224297
Epoch    3 /    5  | Train Loss: 0.6167 Acc: 0.8454  | Val Loss: 1.1244 Acc: 0.7346  | Elapsed time: 3:58:54.970532
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5067 Acc: 0.6823  | Val Loss: 1.0881 Acc: 0.7562  | Elapsed time: 1:21:08.680264
Epoch    2 /    5  | Train Loss: 1.3111 Acc: 0.7191  | Val Loss: 1.0673 Acc: 0.7716  | Elapsed time: 2:41:32.846402
Phase 3:
Epoch    1 /    5  | Train Loss: 1.2929 Acc: 0.7225  | Val Loss: 1.1342 Acc: 0.7500  | Elapsed time: 1:20:42.444327
Epoch    2 /    5  | Train Loss: 1.2441 Acc: 0.7325  | Val Loss: 1.0588 Acc: 0.7747  | Elapsed time: 2:41:30.319693
Epoch    3 /    5  | Train Loss: 0.9636 Acc: 0.7913  | Val Loss: 0.8902 Acc: 0.8148  | Elapsed time: 4:02:22.746229
Epoch    4 /    5  | Train Loss: 0.8768 Acc: 

In [22]:
model_data = torch.load(f"{dataset_dir}/checkpoint.lepidoptera.ta.ep050004.pth", weights_only=False)

In [23]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 119/153 -> 77.78%, genus matched: 138/153 -> 90.20%
Top   3 accuracy: 135/153 -> 88.24%, genus matched: 150/153 -> 98.04%
Top   5 accuracy: 144/153 -> 94.12%, genus matched: 152/153 -> 99.35%
Top  10 accuracy: 145/153 -> 94.77%, genus matched: 153/153 -> 100.00%


In [24]:
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 130/152 -> 85.53%, genus matched: 146/152 -> 96.05%
Top   3 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%
Top   5 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%
Top  10 accuracy: 147/152 -> 96.71%, genus matched: 147/152 -> 96.71%


In [25]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

acraea-terpsicore             : acraea-terpsicore(0.971)  lycaena-panava(0.016)  argynnis-hybrida(0.008)  
athyma-pravara                : neptis-clinia(0.648)  neptis-nata(0.158)  neptis-soma(0.044)  
colias-fieldii                : colias-fieldii(0.998)  colias-eogene(0.001)  colias-erate(0.001)  
danaus-melanippus             : danaus-melanippus(0.926)  danaus-genutia(0.070)  danaus-chrysippus(0.000)  
delias-descombesi             : delias-descombesi(0.944)  delias-agostina(0.008)  delias-pasithoe(0.005)  
euploea-core                  : euploea-core(0.431)  euploea-sylvester(0.267)  euploea-algea(0.144)  
graphium-doson                : graphium-doson(0.982)  graphium-teredon(0.008)  graphium-sarpedon(0.005)  
hypolimnas-bolina             : hypolimnas-bolina(0.999)  hypolimnas-misippus(0.001)  mimathyma-ambica(0.000)  
kallima-inachus               : kallima-inachus(0.997)  doleschallia-bisaltide(0.001)  kallima-albofasciata(0.000)  
lethe-verma                   : lethe-verma(0.

In [20]:
test_top_k(model_data, f"{dataset_dir}/../moth/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

apona-spp                     : apona-caschmirensis(0.382)  polyptychus-trilineatus(0.232)  marumba-dyras(0.101)  
dysphania-percota             : dysphania-percota(0.997)  dysphania-percota-early(0.000)  teliphasa-albifusa(0.000)  
eupterote-undata              : eupterote-undata(0.990)  eupterote-spp(0.005)  eupterote-mollifera(0.002)  
hippotion-rosetta-2           : hippotion-boerhaviae(0.536)  hippotion-rosetta(0.226)  cechetra-minor(0.154)  
hippotion-rosetta             : hippotion-rosetta(0.396)  hippotion-boerhaviae(0.364)  hippotion-spp(0.176)  
----------
Top   1 accuracy: 3/5 -> 60.00%, genus matched: 5/5 -> 100.00%
Top   3 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%


### Model B (resnet-101 + sorted data)
inaturalist data had lots of early stage data mixed in imago class, have been sorted now manually

In [9]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet101", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.tb.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.2218 Acc: 0.5835  | Val Loss: 1.3690 Acc: 0.7037  | Elapsed time: 1:06:34.353595
Epoch    2 /    5  | Train Loss: 0.8291 Acc: 0.8009  | Val Loss: 1.1166 Acc: 0.7407  | Elapsed time: 2:11:51.312474
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5705 Acc: 0.6715  | Val Loss: 1.0159 Acc: 0.7469  | Elapsed time: 1:07:39.699875
Epoch    2 /    5  | Train Loss: 1.3733 Acc: 0.7075  | Val Loss: 0.9844 Acc: 0.7593  | Elapsed time: 2:15:28.483240
Phase 3:
Epoch    1 /    5  | Train Loss: 1.3445 Acc: 0.7138  | Val Loss: 0.9940 Acc: 0.7716  | Elapsed time: 1:07:17.710645
Epoch    2 /    5  | Train Loss: 1.2949 Acc: 0.7240  | Val Loss: 1.0233 Acc: 0.7531  | Elapsed time: 2:14:56.035088
Phase 4:
Epoch    1 /    5 

KeyboardInterrupt: 

### Model C (resnet-152 + sorted data) ***

In [10]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet152", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.tc.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.1977 Acc: 0.5879  | Val Loss: 1.2671 Acc: 0.6944  | Elapsed time: 1:20:03.938434
Epoch    2 /    5  | Train Loss: 0.8228 Acc: 0.8023  | Val Loss: 1.1125 Acc: 0.7284  | Elapsed time: 2:41:38.858001
Phase 2:
Epoch    1 /    5  | Train Loss: 1.5379 Acc: 0.6765  | Val Loss: 1.0099 Acc: 0.7654  | Elapsed time: 1:23:44.811123
Epoch    2 /    5  | Train Loss: 1.3459 Acc: 0.7117  | Val Loss: 1.0590 Acc: 0.7469  | Elapsed time: 2:47:16.850669
Phase 3:
Epoch    1 /    5  | Train Loss: 1.3132 Acc: 0.7188  | Val Loss: 1.0038 Acc: 0.7685  | Elapsed time: 1:24:05.802611
Epoch    2 /    5  | Train Loss: 1.2663 Acc: 0.7291  | Val Loss: 1.0119 Acc: 0.7346  | Elapsed time: 2:48:06.307405
Phase 4:
Epoch    1 /    5  | Train Loss: 1.3079 Acc: 0.7206  | Val Loss: 1.0882 Acc: 0.7469  | Elapsed time: 2:25:26.729687
Epoch    2 /    5  | Train Loss: 1.0291 Acc: 0.7792  | Val Loss: 0.8902 Acc: 0.7901  | Elapsed time: 3:46:25.230717
Epoch    3 /    5  | Train Loss: 0.9

In [9]:
model_data = torch.load(f"{dataset_dir}/checkpoint.lepidoptera.tc.ep050001.pth", weights_only=False)

In [10]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../butterfly/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 113/153 -> 73.86%, genus matched: 133/153 -> 86.93%
Top   3 accuracy: 139/153 -> 90.85%, genus matched: 150/153 -> 98.04%
Top   5 accuracy: 144/153 -> 94.12%, genus matched: 152/153 -> 99.35%
Top  10 accuracy: 146/153 -> 95.42%, genus matched: 153/153 -> 100.00%


reduced top 1 accuracy, but increased top 3 accuracy

In [11]:
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/../moth/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 128/152 -> 84.21%, genus matched: 146/152 -> 96.05%
Top   3 accuracy: 146/152 -> 96.05%, genus matched: 147/152 -> 96.71%
Top   5 accuracy: 148/152 -> 97.37%, genus matched: 148/152 -> 97.37%
Top  10 accuracy: 148/152 -> 97.37%, genus matched: 149/152 -> 98.03%


In [12]:
test_top_k(model_data, f"{dataset_dir}/../butterfly/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

acraea-terpsicore             : acraea-terpsicore(0.997)  danaus-chrysippus(0.001)  argina-astrea(0.001)  
athyma-pravara                : neptis-clinia(0.405)  neptis-nata(0.204)  neptis-harita(0.159)  
colias-fieldii                : colias-fieldii(0.978)  colias-erate(0.018)  colias-nilagiriensis(0.001)  
danaus-melanippus             : danaus-melanippus(0.939)  danaus-genutia(0.059)  danaus-chrysippus(0.000)  
delias-descombesi             : delias-descombesi(0.989)  delias-pasithoe(0.003)  delias-descombesi-early(0.001)  
euploea-core                  : euploea-core(0.441)  euploea-sylvester(0.230)  euploea-godartii(0.145)  
graphium-doson                : graphium-doson(0.874)  graphium-teredon(0.080)  graphium-sarpedon(0.036)  
hypolimnas-bolina             : hypolimnas-bolina(0.989)  hypolimnas-misippus(0.007)  mimathyma-ambica(0.001)  
kallima-inachus               : kallima-inachus(0.999)  kallima-albofasciata(0.001)  kallima-horsfieldii(0.000)  
lethe-verma                  

In [13]:
test_top_k(model_data, f"{dataset_dir}/../moth/my-test", 3, print_preds=True, print_top1_accuracy=True, print_no_match=False)

apona-spp                     : marumba-dyras(0.447)  apona-caschmirensis(0.162)  polyptychus-trilineatus(0.123)  
dysphania-percota             : dysphania-percota(0.995)  pachyodes-haemataria(0.001)  dysphania-percota-early(0.001)  
eupterote-undata              : eupterote-undata(0.996)  eupterote-spp(0.002)  eupterote-mollifera(0.000)  
hippotion-rosetta-2           : cechetra-minor(0.470)  hippotion-boerhaviae(0.209)  hippotion-rosetta(0.182)  
hippotion-rosetta             : hippotion-boerhaviae(0.507)  hippotion-rosetta(0.262)  hippotion-spp(0.056)  
----------
Top   1 accuracy: 2/5 -> 40.00%, genus matched: 3/5 -> 60.00%
Top   3 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%


### Model D (resnet-152 + more species)

In [7]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 6, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet152", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.lepidoptera.td.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 2.2655 Acc: 0.5688  | Val Loss: 1.3073 Acc: 0.6780  | Elapsed time: 2:02:12.433761
Epoch    2 /    5  | Train Loss: 0.8992 Acc: 0.7774  | Val Loss: 1.2307 Acc: 0.7090  | Elapsed time: 3:57:24.550842
Phase 2:
Epoch    1 /    5  | Train Loss: 1.6152 Acc: 0.6551  | Val Loss: 1.0624 Acc: 0.7461  | Elapsed time: 2:04:49.408890
Epoch    2 /    5  | Train Loss: 1.4142 Acc: 0.6921  | Val Loss: 1.0937 Acc: 0.7523  | Elapsed time: 4:00:55.403772
Phase 3:
Epoch    1 /    5  | Train Loss: 1.3906 Acc: 0.6972  | Val Loss: 1.0870 Acc: 0.7554  | Elapsed time: 1:53:22.447964
Epoch    2 /    5  | Train Loss: 1.3428 Acc: 0.7069  | Val Loss: 1.0982 Acc: 0.7461  | Elapsed time: 3:45:43.227653
Phase 4:
Epoch    1 /    5  | Train Loss: 1.3839 Acc: 0.6985  | Val Loss: 1.0711 Acc: 0.7399  | Elapsed time: 1:52:35.319650
Epoch    2 /    5  | Train Loss: 1.0969 Acc: 0.7569  | Val Loss: 0.9380 Acc: 0.7709  | Elapsed time: 3:49:36.904192
Epoch    3 /    5  | Train Loss: 1.0

In [8]:
model_data = torch.load(f"{dataset_dir}/checkpoint.lepidoptera.td.ep060001.pth", weights_only=False)

In [9]:
test_top_k(model_data, f"{dataset_dir}/my-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=True)
test_top_k(model_data, f"{dataset_dir}/my-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/my-test", 10, print_preds=False, print_top1_accuracy=False)

athyma-pravara: 
	 neptis-nata(0.293)  neptis-clinia(0.235)  neptis-harita(0.166)  
eupterote-undata-2: 
	 gunda-javanica(0.209)  episparis-costistriga(0.142)  lymantria-fuliginosa(0.078)  
hypolimnas-missippus: 
	 hypolimnas-misippus(0.676)  hypolimnas-bolina(0.298)  athyma-punctata(0.008)  
unidentified-moth-2: 
	 orgyia-postica-early(0.762)  lymantria-ampla(0.052)  olene-mendosa-early(0.039)  
unidentified-moth-3a: 
	 locastra-muscosalis(0.628)  pyralidae-genera-spp(0.206)  arippara-indicator(0.032)  
unidentified-moth-3b: 
	 locastra-muscosalis(0.613)  aporodes-floralis(0.183)  pyralis-manihotalis(0.055)  
unidentified-moth-4a: 
	 cleora-injectaria(0.434)  cleora-alienaria(0.140)  cusiala-boarmoides(0.097)  
unidentified-moth-4b: 
	 cleora-injectaria(0.722)  boarmiini-genera-spp(0.119)  cleora-spp(0.061)  
vindula-dejone: 
	 vindula-erota(0.729)  cirrochroa-tyche(0.201)  cirrochroa-aoris(0.027)  
Top   1 accuracy: 34/49 -> 69.39%, genus matched: 41/49 -> 83.67%
Top   3 accuracy: 40